## Fala dev! 
#### Nesse notebook nós vamos começar a entender como inicializar um projeto, como funciona um ambiente virtual e como instalar as bibliotecas necessárias para o nosso projeto. 

#### Iremos entender também como usar os tokens e já indicar para você o caminho para a proximas etapas. Então, vamos lá!

## Antes de começar a qualquer projeto, é interessante você criar um ambiente virtual. 
### Mas o que é um ambiente virtual?
Ambiente virtual é um ambiente isolado do seu sistema operacional, onde você pode instalar as bibliotecas necessárias para o seu projeto sem interferir no seu sistema operacional.

Olhe o exemplo abaixo de como instalar um ambiente virtual no seu sistema operacional:

```bash
pip install virtualenv
```

Feito isso, você pode criar um ambiente virtual com o comando abaixo:

```bash
virtualenv nome_do_seu_ambiente
```
Em geral chamamos o ambiente virtual de `env`, ou `venv`, mas você pode chamar do que quiser.

Para ativar o ambiente virtual, você pode usar o comando abaixo:

```bash
cd nome_do_seu_ambiente/Scrips
.\activate
```

Se você observar o seu terminal, você verá que o nome do seu ambiente virtual aparecerá antes do seu nome de usuário.
(Ex: (nome_do_seu_ambiente) C:\Users\seu_nome_de_usuario>)



### Requirements
Bem agora que voçê já sabe o que é um ambiente virtual, vamos instalar as bibliotecas necessárias para o nosso projeto.

Para isso muitas vezes usamos o arquivo `requirements.txt` que contém todas as bibliotecas necessárias para o nosso projeto.

Obs: É importante que na construão do projeto a medida que você for instalando as bibliotecas, você vá atualizando o arquivo `requirements.txt`.

Para instalar as bibliotecas necessárias para o nosso projeto, você pode usar o comando abaixo:

```bash
pip install -r requirements.txt
```

### Tokens

Para você conseguir acessar a API da OpenAI, você precisa de um token. Para conseguir um token, você precisa se cadastrar no site da OpenAI e solicitar um token, mas provavelmente o grade mestre senhor kaiô ja lhe passou seu token no privado.

### Próximos passos
Crie um arquivo `.env` e coloque o seu token lá. 

```bash
OPENAI_API_KEY=seu_token
```

Agora que você já tem o seu ambiente virtual, já instalou as bibliotecas necessárias e já tem o seu token, você já está pronto para começar a codar... ou não.

In [ ]:
## Agora vamos ver como o modelo fuciona via API
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

Deu errado? Não esqueceu de dar pip install nessa nova biblioteca que você quer utilizar?

Já atualizou o requirements.txt hoje irmão?

Para entender mais sobre ChatOpenAI, você pode acessar a documentação da OpenAI [aqui](https://api.python.langchain.com/en/latest/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html).

In [ ]:
# Vamos fazer uma pergunta
question = "Qual é a capital do Brasil?"
answer = llm.invoke(question)
print(answer)

In [ ]:
# Podemos orientar resposta do modelo com um modelo de prompt 
# Esse modelo de prompt é uma string que é usada para orientar a resposta do modelo

from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "Voçê é um ajudante de aluno e está respondendo perguntas de geografia."),
    ("user", "{input}")
])

## Cadeia de comandos
Chains of commands, ou cadeia de comandos, é uma sequência de comandos que são executados em sequência.

Vamos combinar os comandos anteriores no que chamamos de chain.

In [ ]:
chain = prompt | llm

Agora podemos invocá-lo e fazer a mesma pergunta.

In [ ]:
chain.invoke({"input": "Qual é a capital do Brasil?"})

A saída de um ChatModel (e, portanto, desta cadeia) é uma mensagem. No entanto, muitas vezes é muito mais conveniente trabalhar com strings. 

Vamos adicionar um analisador de saída simples para converter a mensagem de chat em uma string.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [ ]:
chain = prompt | llm | output_parser

In [ ]:
chain.invoke({"input": "Qual é a capital do Brasil?"})

Bem, essa é uma aplicação simples de cadeia de comandos. É possível criar cadeias de comandos mais complexas, 

com várias entradas e saídas, e até mesmo cadeias de comandos que chamam outras cadeias de comandos.

Se aventure e crie suas chains pequeno garfanhoto.

mais informações sobre chains [aqui](https://python.langchain.com/v0.1/docs/expression_language/cookbook/multiple_chains/)


# Plus 
E se você quiser streamar a resposta do seu chatbot?


In [ ]:
# Vamos usar o mesmo prompt para fazer outra pergunta e a resposta queremos que seja stream
async for msg in chain.astream({"input": "Qual é a capital do Brasil? Fale brevemente sobre a história do Brasil."}):
    print(msg, end="", flush=True)

### Então quer dizer que você já está streamando a resposta do seu chatbot?
Já pode começar a ir pensando em como você aplicar isso em um projeto real.

# Embeddings

### Dependências

In [ ]:
!pip install bs4

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

#### API KEY

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Dados

Aqui estaremos carregando um texto sobre `Agents` de um blog no GitHub para exemplificação. Este foi um exemplo tirado da própria documentação do LangChain.

In [ ]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [ ]:
len(docs)

In [ ]:
docs[0]

In [ ]:
docs[0].page_content

In [ ]:
len(docs[0].page_content)

# Chunks e Embeddings
Nesse momento, entramos no foco desta etapa do tutorial que são: **chunks** e **embeddings**. Uma breve contextualização:

- `Chunks`: são os pedaços de texto provenientes da divisão do seu documento, basicamente.
- `Embeddings`: representações vetoriais númericas que incorporam semanticamente o conteúdo dos seus documentos (chunks).

Seguiremos o tutorial utilizando as ferramentas do LangChain.

### RecursiveCharacterTextSplitter

A biblioteca fornece uma gama de `text_splitters`, que são classes/ferramentas usadas para conseguirmos realizar a divisão dos nossos textos e gerar os tão famosos chunks. O mais básico e mais usual deles é o `RecursiveCharacterTextSplitter`, recomendado para texto genérico. É parametrizado por uma lista de caracteres. Vamos dar uma olhada em como nós instanciamos esta classe.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

Vamos examinar os parâmetros definidos acima para o uso do **RecursiveCharacterTextSplitter**:

- `chunk_size`: O tamanho máximo do chunk, onde o tamanho é determinado pela length_function. Geralmente, utilizamos a própria função `len`, então será o número máximo de caracteres por chunk. 
- `chunk_overlap`: Sobreposição entre os chunks. A sobreposição de blocos ajuda a mitigar a perda de informações quando o contexto é dividido entre blocos. Controla o quanto cada chunk consecutivo terá em comum com o bloco anterior.
- `length_function`: Função que determina o tamanho do chunk.
- `is_separator_regex`: Se a lista de separadores (padrão ["\n\n", "\n", " ", ""]) deve ser interpretada como regex.

Agora que um ``text_splitter`` foi instanciado, podemos então gerar os nossos chunks. Com os parâmetros que usamos, dados o(s) nosso(s) documento(s), estaremos dividindo ele em chunks de 1000 caracteres no máximo, com sobreposição de 200 caracteres. Vamos ver a mágica acontecendo.

In [ ]:
chunks = text_splitter.split_documents(docs)

In [ ]:
len(chunks), type(chunks)

Se você notar, a variável chunks é uma lista com 66 elementos. Esses elementos, conforme vemos abaixo, são instâncias da classe Document, assim como o seu documento carregado inicialmente. No entanto, agora temos mais porque os chunks são representações de menor tamanho do documento original, mas que o representam em sua totalidade.

**Adendo**:
Se tivéssemos uma string pura, ou um arquivo de texto lido diretamente, para gerarmos chunks dessa string pura precisamos utilizar um outro método:

In [ ]:
string_pura = """ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, 
                quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. 
                Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor 
                incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit 
                in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor 
                sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea
                commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia 
                deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud 
                exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat 
                cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore 
                magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore
                eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do
                eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in 
                reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum 
                lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam quis nostrud exercitation ullamco laboris nisi ut aliquip 
                ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident sunt in culpa qui officia 
                deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam quis nostrud 
                exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat 
                cupidatat non proident sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore 
                magna aliqua. Ut enim ad minim veniam quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore 
                eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do 
                eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in 
                reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum 
                lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam quis nostrud exercitation ullamco laboris nisi ut aliquip 
                ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident sunt in culpa qui officia 
                deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam quis nostrud 
                exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fug ipsum lorem dolor sit amet, consectetur 
                adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. 
                Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit 
                anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation 
                ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat 
                non proident, sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna 
                aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore
                eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit 
                sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure 
                dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est 
                laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco 
                laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non 
                proident, sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. 
                Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat 
                nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod 
                tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit 
                in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit 
                amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo 
                consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident sunt in culpa qui officia deserunt 
                mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam quis nostrud exercitation 
                ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident 
                sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim 
                veniam quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur 
                sint occaecat cupidatat non proident sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et 
                dolore magna aliqua. Ut enim ad minim veniam quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore 
                eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident sunt in culpa qui officia deserunt mollit anim id est laborum. ipsum lorem dolor sit amet, consectetur adipiscing elit sed do eiusmod 
                tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in 
                voluptate velit esse cillum dolore eu fug"""

chunks_2 = text_splitter.create_documents([string_pura]) # passamos dentro de uma lista

In [ ]:
chunks_2

### OpenAI Embeddings

Agora, a ideia é que geremos `embeddings`, isto é, as representações vetoriais semânticas, dos chunks que acabamos de criar. Aqui, estaremos utilizando o modelo de embeddings da OpenAI para gerá-los e faremos a conexão ainda por meio da classe que o LangChain disponibiliza para realizarmos essa requisição de forma fácil, o `OpenAIEmbeddings`.

Logo, temos:

#### 1. Instancie a classe com sua credencial (sua API Token) e sete o modelo que você quer utilizar.

In [ ]:
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY, model="text-embedding-3-small", dimensions=1536)

Considerações:

1. Passamos a chave para conectar com a API. Lembre que a cada requisição para a API da OpenAI, haverá algum custo associado por requisição.
2. O modelo que utilizamos aqui foi o `text-embedding-3-small`, e é válido salientar que ele não é o único modelo de embedding da OpenAI. Em termos de desempenho e custo, ele está no intermedio, ou seja, não é o mais barato nem o mais caro, além de não ser o pior nem o melhor em questão de performance. Os outros que existem vou deixar como pesquisa, além de noções de custo.
3. As incorporações são feitas em um espaço n-dimensional. Por padrão, para esse modelo utilizado, as incorporações são feitas em um espaço dimensional de 1536 dimensões, mas se olhar a documentação da OpenAI, a incorporação pode ser feita para outro número de dimensões e a ideia é a mesma para os outros modelos de embeddings da OpenAI. Vale a pesquisa de quais são.|

#### 2. Gere os embeddings
Quando vamos gerar embeddings, geramos embeddings de textos (conteúdos textuais, strings). Passamos como parâmetro de fato uma lista, mas uma lista de strings. Não podemos passar a lista dos chunks diretamente porque os elementos da lista são do tipo Document, logo, basta fazermos:

In [ ]:
embeddings_agent = embeddings.embed_documents([chunk.page_content for chunk in chunks])

In [ ]:
embeddings_agent[0]

In [ ]:
len(embeddings_agent), len(embeddings_agent[0])
# quantos embeddings, quantas dimensões

E assim geramos embeddings dos nossos chunks/documentos.

## Aquisição externa de Dados

Para trabalhar em projetos reais, muitas vezes é necessário trabalhar com dados externos. Por isso, vamos fazer um exemplo com a API do Youtube integrada com o LangChain.

In [ ]:
!pip install --upgrade --quiet  youtube-transcript-api

In [ ]:
from langchain_community.document_loaders import YoutubeLoader

In [ ]:
loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=xdfVomq3lAc", add_video_info=False
)
loader.load()

In [ ]:
%pip install --upgrade --quiet  pytube

In [ ]:
loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=xdfVomq3lAc", add_video_info=True
)
loader.load()

## Criando Chunks

Muitos textos são muito grandes quando saem de uma transcrição do vídeo. Para facilitar o processamento, é interessante dividir o texto em pedaços menores, nosso chunks que já vimos antes.

### Opção 1: Text Splitter

In [ ]:
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

chunks_split = text_splitter.split_documents(docs)
chunks_split

### Opção 2: Usando CHUNKS por TimeStamp

In [ ]:
from langchain_community.document_loaders.youtube import TranscriptFormat

loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=xdfVomq3lAc",
    add_video_info=True,
    transcript_format=TranscriptFormat.CHUNKS,
    chunk_size_seconds=60,
)
chunks_transcript = loader.load()
chunks_transcript

In [ ]:
len(chunks_transcript), len(chunks_split)

Bem, parece que o chunks por text_splitter dicidiu mais a transcrição do texto. Se isso é melhor ou pior vai depender do problema que se está trabalhando.

### Embeddings

In [ ]:
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY, model="text-embedding-3-small", dimensions=128)
embeddings_agent = embeddings.embed_documents([chunk.page_content for chunk in chunks_split])

In [ ]:
embeddings_agent[0]

### Para amanhã

1. Escolher duas playlists do youtube com temas distintos
2. Faça o Embeddings dos vídeos
3. Compare os embeddings dos vídeos das duas playlists em um gráfico bidimensional

O que significa o posicionamentos desses chunks no gráfico? Como você pode interpretar isso?